In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from IPython.display import display, clear_output

# import pyro
# import pyro.infer
# import pyro.optim
# import pyro.distributions as dist
import torch.distributions.constraints as constraints
# import pyro.poutine as poutine

from datasets import load_dataset

# pyro.set_rng_seed(1)
# pyro.clear_param_store()


#############################
######## TODO ###############
####### ADD LEMMATZATIONN ###
#############################

In [2]:
pip install datasets


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [78]:
from datasets import logging
logging.set_verbosity_error()


ds = load_dataset("armanc/pubmed-rct20k")

Repo card metadata block was not found. Setting CardData to empty.


In [44]:
# =========================
# PubMed LDA Preprocessing for ~600 docs (Gibbs unchanged)
# =========================
import numpy as np
import pandas as pd
import re
from collections import Counter
from datasets import load_dataset, logging as ds_logging

# ---- Config for ~600 abstracts ----
RANDOM_STATE     = 42
SAMPLE_N         = 600
MIN_TOKENS_DOC   = 20          # drop ultra-short docs after cleaning
MIN_DF_ABS       = 5           # absolute min_df for ~600 docs
MAX_DF_FRAC      = 0.45        # prune top ~45% most ubiquitous tokens
PHRASE_MIN_COUNT = 5           # tuned for ~600 docs
PHRASE_THRESHOLD = 5.0

np.random.seed(RANDOM_STATE)
ds_logging.set_verbosity_error()

# ---- 1) Load & rebuild abstracts ----
ds = load_dataset("armanc/pubmed-rct20k")
train_df = ds["train"].to_pandas()

abstracts = (
    train_df.sort_values(["abstract_id", "sentence_id"])
            .groupby("abstract_id")["text"]
            .apply(lambda s: " ".join(s))
            .reset_index()
)

# sample ~600 abstracts
abstracts = abstracts.sample(n=SAMPLE_N, random_state=RANDOM_STATE).reset_index(drop=True)

# ---- 2) Stoplists & markers (APPLIED AFTER LEMMA) ----
# PubMed/trial boilerplate (keep domain signal like hiv/risk/hazard/survival)
PUBMED_STOP = {
    "study","studies","background","objective","objectives","method","methods",
    "result","results","conclusion","conclusions","introduction","aim","aims",
    "patient","patients","participant","participants","subject","subjects",
    "group","groups","arm","arms","trial","cohort","sample","samples","baseline",
    "randomize","randomised","randomized","double","blind","double-blind","placebo",
    "include","including","perform","performed","report","reported","assess","assessed",
    "measure","measured","analyze","analysis","analyses","associate","associated",
    "increase","increased","decrease","decreased","rate","rates",
    "week","weeks","month","months","year","years","daily","usual",
    "based","level","levels","score","scores","change","changes","clinical","life",
    "respectively","data","follow","follow-up","difference","differences","receive","received",
    "effect","effects","primary","event","events","quality","safety","reduction","phase"
}

# Generic glue (lemma forms). We target bare verbs/nouns you flagged.
GLUE_STOP = {
    "this","that","there","from","over","either","only","but","most","into","under",
    "who","have","had","do","did","they","those","among","also","when","per",
    "than","more","both","each","all","will","be","are","is","been","after","before",
    "during","between","time","day","days","mean","range","two"
}

# Domain-generic process verbs/nouns (lemma) you don’t want as anchors
DOMAIN_GLUE = {
    "compare","control","significant","significantly","test","find","program",
    "effective","effectiveness","intervention","interventions","outcome","outcomes",
    "assign","randomly","evaluate","assess","measure","analyze","detect","improve","improvement"
}

# Dataset bracket/marker junk
MARKER_TOKENS = {
    "-lsb-","-rsb-","-lrb-","-rrb-","-lsqb-","-rsqb-","lsb","rsb","lrb","rrb","lsqb","rsqb"
}

# tokens to drop if they contain digits or start with a hyphen (e.g., -mm, -week)
def _bad_token(tok: str) -> bool:
    return any(ch.isdigit() for ch in tok) or tok.startswith("-")

# ---- 3) Tokenization + Lemmatization (scispaCy) with 2-step stopwording ----
# allow letters, hyphen, underscore (underscore marks phrases)
_WORD_RE = re.compile(r"[a-z\-_]+")

def _simple_tokenize(text: str, base_stop):
    text = text.lower()
    toks = _WORD_RE.findall(text)
    toks = [t for t in toks if len(t) > 2 and t not in base_stop and not _bad_token(t)]
    return toks

def tokenize_docs(texts):
    """
    Layer 0: remove markers/digits/hyphen-leading crud
    Layer 1: scispaCy lemma + NLTK English stopwords
    Layer 2: PubMed boilerplate + glue stops (on LEMMAS)
    """
    base_stop = set(MARKER_TOKENS)

    try:
        import spacy
        from scispacy.abbreviation import AbbreviationDetector
        from nltk.corpus import stopwords as _sw
        import nltk; nltk.download("stopwords", quiet=True)
        en_sw = set(_sw.words("english"))

        nlp = spacy.load("en_core_sci_md", disable=["ner","textcat"])
        nlp.add_pipe("abbreviation_detector")

        out = []
        for doc in nlp.pipe(texts, batch_size=64):
            # Expand abbreviations (e.g., PFS -> progression free survival)
            abbr_map = {str(abbr): str(abbr._.long_form) for abbr in doc._.abbreviations}
            txt = doc.text
            for abbr, longf in abbr_map.items():
                txt = re.sub(rf"\b{re.escape(abbr)}\b", longf, txt, flags=re.IGNORECASE)

            d2 = nlp.make_doc(txt)

            # Layer 1: lemma + NLTK stopwords + base filters
            toks_lemma = []
            for t in d2:
                s = (t.lemma_ or t.text).lower()
                if not _WORD_RE.fullmatch(s):
                    continue
                if len(s) < 3:
                    continue
                if s in en_sw:
                    continue
                if s in base_stop or _bad_token(s):
                    continue
                toks_lemma.append(s)

            # Layer 2: PubMed boilerplate + glue + domain-generic process words
            toks = [t for t in toks_lemma if t not in PUBMED_STOP and t not in GLUE_STOP and t not in DOMAIN_GLUE]
            out.append(toks)
        return out

    except Exception:
        # Fallback: simple + NLTK layer + domain stops
        try:
            from nltk.corpus import stopwords as _sw
            import nltk; nltk.download("stopwords", quiet=True)
            en_sw = set(_sw.words("english"))
        except Exception:
            en_sw = set()

        toks0 = [_simple_tokenize(t, base_stop) for t in texts]
        toks1 = [[w for w in tl if w not in en_sw] for tl in toks0]
        toks2 = [[w for w in tl if w not in PUBMED_STOP and w not in GLUE_STOP and w not in DOMAIN_GLUE] for tl in toks1]
        return toks2

tokenized = tokenize_docs(abstracts["text"].tolist())
tokenized = [tl for tl in tokenized if len(tl) >= MIN_TOKENS_DOC]

# ---- 4) Phrase mining (bi -> tri), tuned for ~600 docs ----
try:
    from gensim.models.phrases import Phrases, Phraser
    big = Phrases(tokenized, min_count=PHRASE_MIN_COUNT, threshold=PHRASE_THRESHOLD, delimiter=b"_")
    big_p = Phraser(big)
    tokenized = [big_p(tl) for tl in tokenized]

    tri = Phrases(tokenized, min_count=PHRASE_MIN_COUNT, threshold=PHRASE_THRESHOLD, delimiter=b"_")
    tri_p = Phraser(tri)
    tokenized = [tri_p(tl) for tl in tokenized]
except Exception:
    pass

# optional: hard-seed a few must-have phrases that often help interpretability
MUST_PHRASES = [
    ("blood", "pressure"), ("weight", "loss"),
    ("hazard", "ratio"), ("odds", "ratio"),
    ("overall", "survival"), ("progression-free", "survival"),
    ("adverse", "events"), ("ejection", "fraction"),
    ("knee", "pain"), ("viral", "load"), ("cd4", "count"), ("smoking", "cessation")
]
# join where present
for tl in tokenized:
    j = 0
    while j < len(tl) - 1:
        if (tl[j], tl[j+1]) in MUST_PHRASES:
            tl[j:j+2] = [tl[j] + "_" + tl[j+1]]
        else:
            j += 1

# ---- 5) Document-frequency filtering (no top-N cap) ----
D_pre = len(tokenized)
df = Counter()
for tl in tokenized:
    df.update(set(tl))

min_df = max(MIN_DF_ABS, 1)
max_df = int(MAX_DF_FRAC * D_pre)
keep = {w for w, c in df.items() if (c >= min_df and c <= max_df)}

tokenized = [[w for w in tl if w in keep] for tl in tokenized]

# rebuild vocab (stable order)
vocab_list = sorted(keep)
vocab = {w: i for i, w in enumerate(vocab_list)}
i2word = {i: w for w, i in vocab.items()}

print(f"Docs after cleaning: {len(tokenized)} | Vocab size: {len(vocab)} | min_df={min_df}, max_df={max_df}")

# ---- 6) Build words/docs for your Gibbs (names unchanged) ----
words, docs = [], []
for doc_id, doc in enumerate(tokenized):
    for w in doc:
        if w in vocab:
            words.append(vocab[w])
            docs.append(doc_id)

words = np.array(words, dtype=int)
docs  = np.array(docs,  dtype=int)

D = int(docs.max()) + 1 if docs.size else 0
W = int(words.max()) + 1 if words.size else 0
N = len(docs)

print(f"#docs={D}, vocab size={W}, total tokens={N}")

# (Optional quick sanity)
for i in range(min(2, len(tokenized))):
    print("Sample tokens:", tokenized[i][:25])


Repo card metadata block was not found. Setting CardData to empty.


Docs after cleaning: 600 | Vocab size: 1911 | min_df=5, max_df=270
#docs=600, vocab size=1911, total tokens=46378
Sample tokens: ['counselling', 'older', 'people', 'chronic', 'condition', 'live', 'home', 'useful', 'support', 'paper', 'development', 'testing', 'instrument', 'counselling', 'questionnaire', 'developed', 'tested', 'people', 'older', 'component', 'conducted', 'revealed', 'four', 'factor', 'model']
Sample tokens: ['pharmacokinetics', 'doses', 'investigated', 'application', 'two-way', 'cross-over', 'conducted', 'seven', 'healthy', 'postmenopausal', 'women', 'age', 'allocated', 'applied', 'upper', 'serum', 'total', 'free', 'sex', 'concentrations', 'serum', 'pharmacokinetic', 'parameters', 'auc', 'cmax']


In [45]:
# Initialize our latent assignments Z and set up per-X-topic arrays to keep track of assignment totals

np.random.seed(0)

# T = 5        # Only two topics ("HCI" and "Graph theory")
# beta = .01     # LDA parameters: control sparsity of documents & topics
# alpha = .001

# T = 20
# beta = 0.08
# alpha = 0.06

T = 18
beta = 0.07
alpha = 0.1

a = np.zeros((D,T),dtype=int)   # total times a token in document d has been assigned to topic t
b = np.zeros((W,T),dtype=int)   # total times word w has been assigned to topic t
c = np.zeros((T,),dtype=int)    # total assignments to topic t

z = np.random.randint(T,size=N);  # assign every token to a topic at random
for n in range(N):
    a[docs[n],z[n]] += 1             # and each document ID
    b[words[n],z[n]] += 1             # count up number for each word ID
    c[z[n]] += 1

In [48]:
def gibbs_collapsed(maxIter):
    for it in range(maxIter):             # for each iteration 
        if it%10 ==0 : print(it)
        for i in range(N):                # run through all the words & sample z[i]:
            t = z[i];
            a[docs[i],t] -= 1                # remove token i's assignment from our count vectors
            b[words[i],t] -= 1            
            c[t]      -= 1

            # Compute topic probability distribution given current counts
            probs = (beta + b[words[i],:])/(c[:] + beta*W) * (alpha + a[docs[i],:])

            # Now, normalize and draw a sample from the distribution over topics:
            cumprobs = np.cumsum(probs); cumprobs /= cumprobs[-1] 
            t = np.where(cumprobs>np.random.rand())[0][0]

            z[i] = t;                      # now add this assignment back into our count vectors
            a[docs[i],t] += 1
            b[words[i],t] += 1
            c[t]      += 1

In [49]:
# Helper function for visualizing the learning process
def print_topics():
    '''Print the top 8 words in each topic given the current assignments'''
    for t in range(T):
        isort = np.argsort(-b[:,t])  # find the most likely words for topic t
        xsort = b[isort,t]           # then print topic, % tokens explained, & top 8 words
        print('[{}] ({:.3f}) {}'.format(t, 1.*c[t]/N, list(i2word[isort[ww]] for ww in range(8) if xsort[ww]>0)))

In [50]:
print_topics();       # before running any iterations: random assignment

[0] (0.054) ['treatment', 'compared', 'care', 'total', 'using', 'therapy', 'health', 'use']
[1] (0.054) ['treatment', 'compared', 'risk', 'one', 'therapy', 'use', 'women', 'men']
[2] (0.054) ['treatment', 'compared', 'using', 'efficacy', 'risk', 'controlled', 'pain', 'therapy']
[3] (0.057) ['treatment', 'risk', 'care', 'women', 'compared', 'health', 'therapy', 'using']
[4] (0.056) ['treatment', 'care', 'compared', 'health', 'pain', 'therapy', 'women', 'included']
[5] (0.058) ['treatment', 'compared', 'care', 'risk', 'one', 'total', 'use', 'using']
[6] (0.056) ['treatment', 'compared', 'therapy', 'total', 'care', 'surgery', 'using', 'risk']
[7] (0.055) ['treatment', 'compared', 'women', 'adverse', 'lower', 'efficacy', 'children', 'surgery']
[8] (0.054) ['treatment', 'compared', 'care', 'treated', 'period', 'lower', 'greater', 'efficacy']
[9] (0.054) ['treatment', 'compared', 'total', 'pain', 'care', 'therapy', 'risk', 'using']
[10] (0.055) ['treatment', 'compared', 'risk', 'one', 'women

In [51]:
gibbs_collapsed(2600)            # run a bunch of iterations
print_topics();

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [36]:
gibbs_collapsed(2500)            # run a bunch of iterations
print_topics();

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2